<a href="https://colab.research.google.com/github/safdarjung/projects/blob/main/eng2ger_FairSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# data set contains language translations. All sentences are tokenised
! git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

Cloning into 'multi30k-dataset'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 313 (delta 17), reused 21 (delta 16), pack-reused 281
Receiving objects: 100% (313/313), 18.21 MiB | 10.70 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Submodule 'scripts/subword-nmt' (https://github.com/rsennrich/subword-nmt.git) registered for path 'scripts/subword-nmt'
Cloning into '/content/multi30k-dataset/scripts/subword-nmt'...
remote: Enumerating objects: 597, done.        
remote: Counting objects: 100% (21/21), done.        
remote: Compressing objects: 100% (17/17), done.        
remote: Total 597 (delta 8), reused 12 (delta 4), pack-reused 576        
Receiving objects: 100% (597/597), 252.23 KiB | 13.27 MiB/s, done.
Resolving deltas: 100% (357/357), done.
Submodule path 'scripts/subword-nmt': checked out '80b7c1449e2e26673fb0b5cae993fe2d0dc23846'


In [2]:
%%capture
# W and B -- For Logging
! pip install wandb

# Sacremoses -- For Tokenizing
! pip install sacremoses

# fairseq -- For training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# PreProcessing and binarizing to build vocabularies
'''
BINARIZATION
hello: [1, 0, 0, 0]
how: [0, 1, 0, 0]
are: [0, 0, 1, 0]
you: [0, 0, 0, 1]

preprocessing of source language to target language
specifying the training testing and validation data - all tokenised
path for storing the learned vocabulary and binarised dataset
Words occurring fewer than this number of times in the source data will be discarded
for both src and trgt
'''

! fairseq-preprocess --source-lang de --target-lang en \
  --trainpref multi30k-dataset/data/task1/tok/train.lc.norm.tok \
  --validpref multi30k-dataset/data/task1/tok/val.lc.norm.tok \
  --testpref  multi30k-dataset/data/task1/tok/test_2018_flickr.lc.norm.tok \
  --destdir data-bin/multi30k.tokenized.de-en \
  --thresholdsrc 2 \
  --thresholdtgt 2

2024-04-28 11:29:21.423270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:29:21.423317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:29:21.519367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:29:21.717436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 11:29:23.593432: W tensorflow/comp

In [5]:
'''
TRAINING THE MODEL
train a model - directory containing the preprocessed training data in the Fairseq format
using the Transformer architecture, a popular choice for sequence-to-sequence tasks {other archs can also be be used}
dropouts for attention block, feed fwd net block and activation block
dimentions for encoder's token embeddings
encoder's feedforward network inner-layer embeddings.
no. of encoders
no. of attention heads in encoder
Indicates that the encoder should use learned positional embeddings - we can also set to supply the positional embeddings instead of learning them from the data itself.
dimentions for decoder's token embeddings
decoder's feedforward network inner-layer embeddings.
no. of decoders
no. of attention heads in decoder
Indicates that the decoder should use the learned positional embeddings
max training epochs
optimiser - adam
learning rate
batch size
 random seed
 project name
'''

! fairseq-train data-bin/multi30k.tokenized.de-en \
  --arch transformer \
  --dropout 0.1 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 256 \
  --encoder-ffn-embed-dim 512 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 256 \
  --decoder-ffn-embed-dim 512 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --decoder-learned-pos \
  --max-epoch 10 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --wandb-project "Multi 30K En to De Translation"

2024-04-28 11:29:43.389190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:29:43.389233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:29:43.390554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:29:43.397420: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 11:29:44.455842: W tensorflow/comp

In [6]:
'''
used to generate translations using a trained model with Fairseq.
specifying the best_checkpoint{lowest validation loss} as the checkpoint for the testing
batch_size
beam_width for BeamSearch{finds out the 5 most likely words} used during decoding to generate a sentence
setting the random seed
scoring method - BLEU (Bilingual Evaluation Understudy)
project name
'''
# ckpt_best, beam=5
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --seed 1 \
    --scoring bleu \
    --wandb-project "Multi 30K En to De Translation"

Streaming output truncated to the last 5000 lines.
D-279	-0.3453711271286011	girl playing an outdoor game outside .
P-279	-0.2622 -0.2404 -1.3185 -0.4809 -0.0866 -0.2422 -0.1320 -0.0001
S-342	ein radfahrer fährt einer <unk> hinauf .
T-342	a cyclist riding up a grassy dirt trail .
H-342	-1.087695598602295	a bicyclist rides a stone mountain bike .
D-342	-1.087695598602295	a bicyclist rides a stone mountain bike .
P-342	-0.0765 -0.8522 -1.1140 -0.1826 -1.5214 -3.0896 -0.6465 -2.3064 -0.0000
S-281	dieser surfer versucht , nicht <unk> .
T-281	this surfer is trying to avoid wiping out .
H-281	-0.7505254745483398	this surfer is attempting to make <unk> .
D-281	-0.7505254745483398	this surfer is attempting to make <unk> .
P-281	-0.0045 -0.0862 -0.1862 -0.9359 -0.3011 -1.9706 -2.4452 -0.8251 -0.0000
S-232	mann im wasser auf einem winzigen segelboot
T-232	man in the water on a tiny sail boat
H-232	-0.7192732691764832	man on a tiny boat in the water .
D-232	-0.7192732691764832	man on a tiny boat 

In [7]:
# ckpt_best, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2024-04-28 11:50:48.563071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:50:48.563122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:50:48.564565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:50:48.571943: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 11:50:49.685358: W tensorflow/comp

In [8]:
! python fairseq/scripts/average_checkpoints.py --inputs checkpoints --num-epoch-checkpoints 5 --output checkpoints/averaged.pt
'''
AVERAGING CHECKPOINTS
the averaged checkpoint acts as an ensemble of the specified no. of checkpoints
'''

2024-04-28 11:56:58.226836: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:56:58.226889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:56:58.228860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:56:58.239202: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 11:56:59.289990: W tensorflow/comp

'\nAVERAGING CHECKPOINTS\n\n\n'

In [10]:
# ckpt_avg, beam=10
! fairseq-generate data-bin/multi30k.tokenized.de-en \
    --path checkpoints/averaged.pt \
    --batch-size 128 \
    --beam 10 \
    --seed 1 \
    --scoring bleu \
    --quiet \
    --wandb-project "Multi 30K En to De Translation"

2024-04-28 11:59:31.917272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:59:31.917316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:59:31.918573: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:59:31.925568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 11:59:32.986669: W tensorflow/comp

In [11]:
'''
EXPORTING THE MODEL
english_dict, german_dict, avg_checkpt
'''
! mkdir -p trained_model
! cp data-bin/multi30k.tokenized.de-en/dict.de.txt trained_model/dict.de.txt
! cp data-bin/multi30k.tokenized.de-en/dict.en.txt trained_model/dict.en.txt
! cp checkpoints/averaged.pt trained_model/model.pt


In [12]:
!zip -r trained_en_de_model.zip trained_model/

  adding: trained_model/ (stored 0%)
  adding: trained_model/model.pt (deflated 12%)
  adding: trained_model/dict.de.txt (deflated 63%)
  adding: trained_model/dict.en.txt (deflated 58%)


In [13]:
! fairseq-interactive \
  --path trained_model/model.pt \
  --source-lang de --target-lang en \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  trained_model/

2024-04-28 12:06:45.512491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 12:06:45.512544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 12:06:45.513953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 12:06:45.521378: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-28 12:06:46.551847: W tensorflow/comp